## Setting up

In [1]:
from pathlib import Path
import pandas as pd
import random

In [2]:
NB_DIR = Path.cwd()

In [ ]:
#Insert relevant directories here
LOCAL_DATA = 
ROIS_DF =
PY_DIR = 

In [6]:
pd.set_option('display.max_column', 400)

In [9]:
# Loading data
step3_all = pd.read_csv(LOCAL_DATA/'adni_sMCI_vs_cAD.csv')
step3_Train = pd.read_csv(LOCAL_DATA/'Training_sMCI_and_cAD.csv')
step3_Test = pd.read_csv(LOCAL_DATA/'Final_Test_ADNI_AIBL_sMCI_and_cAD.csv')

In [15]:
step3_Train.DX2.unique(), step3_Test.DX2.unique()

(array(['sMCI', 'cAD'], dtype=object), array(['sMCI', 'cAD'], dtype=object))

In [17]:
len(step3_Train.loc[step3_Train.DX2 == 'sMCI'].RID.unique()),len(step3_Train.loc[step3_Train.DX2 == 'cAD'].RID.unique())

(218, 213)

In [18]:
len(step3_Test.loc[step3_Test.DX2 == 'sMCI'].RID.unique()),len(step3_Test.loc[step3_Test.DX2 == 'cAD'].RID.unique())

(61, 60)

## Functions

In [ ]:

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf


class Mixed_Models(object):
    '''
    This class contains every things related to Mixed effect models, the initial values are:
    data: DataFrame
    roi: Dependent variable for mixed model
    formula: fixed effect formula
    re_formuls: random effect formula
    group: the group of dependent variables, defult is RID
    lable: for plotting, defult is 'DX', need to be fixed in each step, based on input data
    lable_int: intigers maped to lables
    '''
    def __init__(self,
                 data,roi,age, formula, re_formula, label, label_int, group='RID'):
        
        self.data =data
        self.roi = roi
        self.age = age
        self.formula = formula
        self.re_formula = re_formula
        self.group = group
        self.label = label
        self.label_int = label_int
   #_________________________________________#

    def mixed_model(self, plot_rs = True):
        '''
        This function uses mixedlm function from statsmodels.formula and run it based on input informations.
        It plot the residuals if plot_rs = True
        '''
        lme = smf.mixedlm(f"{self.roi} ~ {self.formula}", self.data, groups = self.data[self.group], re_formula = f"~  { self.re_formula}")
        print(f"formula: {self.roi} ~ {self.formula}")
        print(f"reformula: ~ {self.re_formula}")
        lmef = lme.fit()
        
        if plot_rs:
            print(lmef.summary())
            df = self.data[[self.age, 'RID',f'{self.label}']]

            df["residuals"] = lmef.resid.values
            df["predicted"] = lmef.fittedvalues
            df=df.sort_values(by= self.age)
            plt.figure(figsize=(12,12))
            ax = sns.residplot(x = self.age, y = "residuals", data = df, lowess=True )
            ax.set(ylabel='Observed - Prediction')
            plt.show()

            ax = sns.lmplot(x =self.age, y="residuals", data=df, hue=f'{self.label}', height=12)
            ax.set(ylabel='Residuals') 
            plt.show()
            sns.lmplot(x = "predicted", y = "residuals", hue=f'{self.label}',data = df, height=12)
            plt.show()
        return lmef
    
    #_________________________________________#
        
    def get_DXs(self, ID):
        
        '''This function gets data and one RID, returns its label'''
        
        return self.data[self.data[self.group]==ID][f'{self.label_int}']
    
    #_________________________________________#
    
    def get_visits(self, ID):
        
        '''This function get df and one RID, returns sub-df related to RID'''
        return self.data[self.data[self.group]==ID]
    
    #_________________________________________#
    
    def data_plot(self, lmef,step1= False, step2=False, step3 = False):
        '''
        This function plots all subject trajectories with their fixed and random effect lines.
        Input is the mixed model outputs.
        '''
        
        fe_line =  lmef.fe_params.Intercept
        for cov in lmef.fe_params.index.drop(['Intercept']):
            fe_line += lmef.fe_params[cov]*self.data[cov]
        
        
        
        
        if step1: #healthy & unhealthy
            colormap = {1.:'dodgerblue',2.: 'purple'}
            textstr = f'Number of subjects: {len(self.data.RID.unique())}\n Blue: Healthy\n Purple: UnHealthy'
        elif step2: # sMCI & mci->AD 
            colormap = {3.:'dodgerblue',4.: 'purple'}
            textstr = f'Number of subjects: {len(self.data.RID.unique())}\n Blue: sMCI\n Purple: cAD \n Gray: Random effect\n Black: Fixed effect'
        elif step3: #sMCI & mci->AD + AD
            colormap = {1.:'dodgerblue',2.: 'purple'}
            textstr = f'Number of subjects: {len(self.data.RID.unique())}\n Blue: sMCI\n Purple: cAD & AD'
        else:
            colormap = {1.:'dodgerblue', 2.:'green', 3.:'fuchsia',4.: 'salmon',
                    5.: 'purple', 6.: 'red'}#fuchsia, crimson
            textstr = f'Number of subjects: {len(self.data.RID.unique())}\n\n Blue: CN\n Green: CN-MCI\n Pink: MCI\n Salmon: MCI-AD\n Purple: AD\n Red: CN-AD'
            
            
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        f, ax = plt.subplots(figsize=(20,12))
        plt.title(f"Age vs {self.roi}, Trajectories, Randoms and Fixed Effects ", fontsize= 20)
        plt.xlabel('Age at Scan', fontsize=20)
        plt.ylabel(self.roi, fontsize=20)
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=15,
                verticalalignment='top', bbox=props)
        
        for ID in self.data.RID.unique():
            DXs = self.get_DXs( ID)
            if max(DXs) == 0:
                color = 'black'
            else:
                color = colormap[max(DXs)]
                
      
            visits = self.get_visits(ID)
            visits1 = visits.sort_values(by=self.age)
            ax.plot(visits1[self.age], visits1[self.roi], linestyle='-', marker='o', color=color)

            re=lmef.fittedvalues.loc[ID]
     
            
            fit_plot= pd.DataFrame(list(zip(visits[self.age].values, re.values)), columns=[self.age, 'fit_val'])
            fit_plot= fit_plot.sort_values(by=self.age)
            ax.plot(fit_plot[self.age], fit_plot.fit_val, linestyle='-', color='grey', linewidth=0.8)
            

        sort_fe_line = pd.DataFrame(list(zip(self.data[self.age] , fe_line)), columns=[self.age, 'fe_line'])
        sort_fe = sort_fe_line.sort_values(by=self.age)
        plt.plot( sort_fe[self.age],sort_fe.fe_line,'k', linewidth= 5, label= 'Fixed Effect')
        plt.show()



In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

class Features:
    
    '''
    This class contains functions to prepare features that we need for machine learning step.
    Initial values for this class are: 
    data: the same DataFrame for mixed model
    lmef : mixed effect output
    roi: dependent variable
    group: the group of dependent variables, defult is RID
    rid_info: list of columns from data, which we need their information in features df 
        such as Gender, education, FS,  ... ) It should always contain at least the lable for training.
    
    '''
    
    def __init__(self,
                 data,
                 lmef,
                 roi,
                 age,
                 group='RID',
                 rid_info=['DX','DXN','SEX', 'PTEDUCAT']):
        

        self.data = data
        self.lmef = lmef
        self.roi = roi
        self.age = age
        self.rid_info = rid_info
        self.group = group

    def features_per_subject(self, rid, plot=True, debug=False):
        '''
        Derive features for an ROI from output of linear mixed effect model for one subject(rid). 
        It plots the subject-trajectory and fitted value of that

        '''
        features = [rid]
        cols = ['RID']
        re = self.lmef.random_effects
        re_df = pd.DataFrame.from_dict(re, orient='index')
        bi = re_df.iloc[re_df.index == rid]


            
        for col in bi.columns:
            features.append(bi[col].values[0])
            cols.append(col)

      

        fit_val = pd.DataFrame(self.lmef.fittedvalues)
        fit_i = [i[0] for i in fit_val.loc[fit_val.index == rid].values]

        rid_data = self.data.iloc[self.data.index == rid]
        rid_val = [rid_data[col].values for col in bi.columns.drop('Group')]
        rid_col = [col for col in bi.columns.drop('Group')]
        rid_val = rid_val + [rid_data[col].values for col in self.rid_info]

        rid_col = (rid_col) + self.rid_info

        rid_val.append(fit_i)
        rid_col.append('fit_val')
        

        df_info = pd.DataFrame(np.array(rid_val).T, columns=rid_col)
        if debug: print('DataFrame for all output\n', df_info, '\n')

        sort_age = df_info.sort_values(by=self.age)
        if debug: print('sort age', sort_age[self.age])

        age_min = rid_data[self.age].min()
        age_max = rid_data[self.age].max()
        age_mean = rid_data[self.age].mean()

        if debug:
            print('min age', age_min, 'max age', age_max, 'mean age', age_mean)

        fe = self.lmef.fe_params
        fe_min = fe.Intercept
        fe_max = fe.Intercept
        
        for cov, val in zip(fe.index, fe.values):
            if cov != 'Intercept':
                fe_min += val * rid_data.loc[rid_data[self.age] == age_min][cov]
                fe_max += val * rid_data.loc[rid_data[self.age] == age_max][cov]
        if debug: print(fe_min.values, '\n', fe_max.values)
        if debug:
            print(rid_data.loc[rid_data[self.age] == age_min][self.roi].values)
        v_devi1 = (
            rid_data.loc[rid_data[self.age] == age_min][self.roi].values[0] -
            fe_min.values[0])
        if debug: print('v_dev1', v_devi1)
        v_devi2 = (
            rid_data.loc[rid_data[self.age] == age_max][self.roi].values[0] -
            fe_max.values[0])
        if debug: print('v_dev2', v_devi2)
        if age_max != age_min:
            max_change = (rid_data[self.roi].max() - rid_data[self.roi].min())/(age_max - age_min)
        else:
            max_change = 0

        if debug: print(f'max change in {self.roi} = ', max_change)

        features.append(v_devi1)
        cols.append('dev_bl')

        features.append(v_devi2)
        cols.append('dev_last')

        features.append(max_change)
        cols.append('max_change')

        features.append(age_mean)
        cols.append('AGE_mean')
        features.append(age_max)
        cols.append('Max_AGE')
        
        for col in (self.rid_info):
            if debug:print(rid_data[col].values[-1])
            features.append(rid_data[col].values[-1])
            cols.append(col)
        if debug:
            print('Features values', features, '\n Features cols', cols, '\n')
        df_feat = pd.DataFrame([features], columns=cols)
        if debug: print('df_features \n', df_feat)
            
        '''
        PLOT
        '''
        
        if plot:
            sort_rid = rid_data.sort_values(by=self.age)
            plt.plot(sort_rid[self.age],
                     sort_rid[self.roi],
                     'b-o',
                     label=f'RID= {rid}')
            re_bl = (fe.Intercept + bi.Group).values
            if debug: print('re base line', re_bl)
            re_line = []
            re_part = []
            for cov in fe.index.drop('Intercept'):
                if cov not in bi.columns:

                
                    re_bl += fe[cov] * sort_rid[cov].values[0]
                else:
                    re_part.append(
                        (fe[cov] + bi[cov].values[0]) * sort_rid[cov].values)

            if debug: print('\n re parts, list of list \n ', re_part)
            if debug:
                print('\n summation of elements of list of list \n ',
                      sum(map(np.array, re_part)))

            re_line.append(re_bl + sum(map(np.array, re_part)))
            if debug: print('re_values for plot', re_line[0])
            plt.plot(sort_rid[self.age], re_line[0], 'k', label='random estimate')
            plt.plot(sort_age[self.age],
                     sort_age.fit_val,
                     'r--',
                     label='fited value')
            #plt.title(f'formula: {.formula}, re_formula: {MM.reformula}')
            plt.legend()
            plt.title(f"fixed ,{set([fe.index[i] for i in range(len(fe.index))]) -set(['Intercept'])}")

        return df_feat

    #________________________________________________

    def features(self):
        '''
        This function returns a data frame with output parameters of mixed effect model's which
        we need for ML part.

        '''
        features = []
        for rid in self.data[f'{self.group}'].unique():
            rid_feat = self.features_per_subject(rid, plot=False)
            features.append(rid_feat)
        df_features = pd.concat(features)

        return df_features
    



In [22]:
columns = [ 
    'Ventricles',
    'Hippocampus',
    'Amygdala',
    'Entorhinal',
    'inferiortemporal_thickness',
    'middletemporal_thickness',
    'fusiform_thickness',
    'frontalpole_thickness',
    'temporalpole_thickness',
    'superiorparietal_thickness',
    'superiortemporal_thickness',
    'superiorfrontal_thickness',
    'TotalGrayVol',
    'inferiorparietal_thickness',
    'parahippocampal_thickness',
    'CortexVol',
    'insula_thickness',
]
step3_Test[columns].head()

,Ventricles,Hippocampus,Amygdala,Entorhinal,inferiortemporal_thickness,middletemporal_thickness,fusiform_thickness,frontalpole_thickness,temporalpole_thickness,superiorparietal_thickness,superiortemporal_thickness,superiorfrontal_thickness,TotalGrayVol,inferiorparietal_thickness,parahippocampal_thickness,CortexVol,insula_thickness
RID,,,,,,,,,,,,,,,,,
102,46318.223694,4329.593568,1493.128494,4.103759,2.924991,3.018105,2.984368,3.102447,4.165835,2.353487,2.891254,3.135509,325366.829679,2.591670,3.574764,231047.854742,3.295422
102,58107.789057,4229.537273,1256.621667,3.749320,2.574542,2.677604,2.535473,2.964563,3.706882,2.208771,2.491688,2.923473,304924.332323,2.440493,3.255563,213761.836142,3.068973
102,53144.645799,4022.003891,1422.637846,3.688645,2.640329,2.700964,2.673341,2.940810,3.696056,2.295381,2.618769,3.003466,308362.373522,2.455054,3.274978,217371.087476,3.237249
102,55763.737917,3945.981951,1435.701143,3.878812,2.604345,2.653675,2.616509,2.767877,3.830834,2.265794,2.567179,2.851670,306852.030345,2.398241,3.288206,215912.803527,2.948979
102,49574.313827,4138.736448,1381.054014,4.039362,2.772703,2.847133,2.900106,3.236049,4.042714,2.294604,2.702295,3.101270,313713.851634,2.477663,3.286340,222848.999996,3.153572


## Remove AD for Train and Test set

In [27]:
def main_step3(train_set,test_set, roi, age,formula, re_formula,rid,group = 'RID',label= 'DX2', label_int = 'DX2N', linear = True):
    '''
    This function gets the train and test set, removes the scans labeled Dementia, uses mixed efect models to extract the features and save them as a detaframe.
    Separately adds the test set to the train set and extracts the features for test set. 
    '''
    
    data = pd.concat([train_set,test_set], sort=False)
    df = data.loc[data.DX != 'Dementia']
    print(f'Scans Labes : {df.DX.unique()}')
    
    df = df[[roi, age,group, label, label_int,'AGE2', 'SEX']].dropna()
    
    print(f'***************************{roi}*****************************')
    
    #--------------------------------
    #           TEST SET
    #--------------------------------
    print('==============================\n')
    
    LME = Mixed_Models(df,
                        roi,
                        age,
                        formula,
                        re_formula,
                        label,
                        label_int,
                        group)
    lme_ = LME.mixed_model(plot_rs = False)
    
    Feat_DF = Features(df,
                       lme_,
                       roi,
                       age,
                       group,
                       rid_info=[label, label_int, 'SEX'])
    
    df_feature = Feat_DF.features() 
    df_feature=df_feature.set_index('RID')
    df_test = df_feature.loc[test_set.index.unique()]

    if linear:
        X_tst = df_test[['Group', age, 'dev_bl', 'dev_last', 'max_change', 'SEX', 'AGE_mean', 'Max_AGE']]
    else:
        X_tst = df_test[['Group', age,'AGE2' ,'dev_bl', 'dev_last', 'max_change', 'SEX', 'AGE_mean', 'Max_AGE']]
    y_tst = df_test[label]
    
    
    #-------------------------------------
    #             Train SET
    #-------------------------------------
    
    train_ = df.loc[train_set.index]
    LME = Mixed_Models(train_,
                        roi,
                        age,
                        formula,
                        re_formula,
                        label,
                        label_int,
                        group)
    lme_ = LME.mixed_model(plot_rs = False)

    Feat_DF = Features(train_,
                       lme_,
                       roi,
                       age,
                       group,
                       rid_info=[label, label_int, 'SEX'])
    #Feat_DF.features_per_subject(rid, plot=True, debug=False)
    df_train = Feat_DF.features() 
    

    return df_test,df_train 

In [49]:
import pickle

f = open(ROIS_DF/"TRAIN_Table_of_Features_Vent_Hipp_Linear_sMCI_cAD.pkl","wb")
pickle.dump(Features_Train,f)
f.close()
f = open(ROIS_DF/"TEST_Table_of_Features_Vent_Hipp_Linear_sMCI_cAD.pkl","wb")
pickle.dump(Features_Test,f)
f.close()

In [50]:
#---------------------------------

f = open(ROIS_DF/"TRAIN_Table_of_Features_Vent_Hipp_NonLinear_sMCI_cAD.pkl","wb")
pickle.dump(N_Features_Train,f)
f.close()

f = open(ROIS_DF/"TEST_Table_of_Features_Vent_Hipp_NonLinear_sMCI_cAD.pkl","wb")
pickle.dump(N_Features_Test,f)
f.close()